\section{JAGS}
\subsection{JAGS and its relation to R}
JAGS is a system that automatically builds Markov chain Monte Carlo (MCMC) samplers for complex hierarchical models. 

JAGS takes a user's description of a hierarchical model for data, and return an MCMC sampleof the posterior distribution. Packages rjags and runjags in R. 

\begin{minted}{R}
install.packages("rjags")
install.packages("runjags")
\end{minted}

\subsection{A complete example}
We will consider the Bayesian estimation of the bias of a coin. 

A parameterized model consists of a likelihood function, which specifies the probability of data given the parameter values. 

Bias of coin, the likelihood function is the Bernoulli distribution. 

To indicate that the data $y_i$ come from a Bernoulli distribution that has parameter $\theta$, we write:
\[
y_i \sim \textrm{dbern}(\theta)
\]
To indicate that the prior distribution is a beta density with shape constanes $A$ and $B$, we write:
\[
\theta \sim \textrm{dbeta}(A, B)
\]
When describing a model, the description logically flows from data to likelihood (with its parameters) to prior. 

\section{STAN}
\subsection{Introduction}




\subsection{Simon Jackman's bayesian model examples }
\subsubsection{Undervoting}
Data 
\begin{minted}{R}
undervote <- tribble(
  ~survey, ~race, ~n, ~undervote,
  "VNS", "black", 6537, 26,
  "VNS", "white", 44531, 91,
  "ANES", "black", 1101, 10,
  "ANES", "white", 9827, 57
  )
\end{minted}

We are interested in analyzing the difference in proportions for each of these surveys independently. We will model the proportions of each race and survey,
\[
y_i \sim \textrm{Binomial}(n_i, \pi_i)
\]
where $i \in \{\textrm{(VNS, black), (VNS, white), (ANES, black), (ANES, white)}\}$ 

We will model the proportions independently by assigning them identical independent uninformative priors,
\[
\pi_i \sim \textrm{Beta}(1,1)
\]
The racial differences in undervoting in each survey are auxiliary quantities 
\[
\delta_{VNS} = \pi_{VNS, black} - \pi_{VNS, white}
\]
\[
\delta_{ANES} = \pi_{ANES, black} - \pi_{ANES, white}
\]
\begin{minted}{R}
undervote_mod <- stan_model("stan/undervote.stan")

data {
  int n[4];
  int y[4];
  vector[4] pi_a;
  vector[4] pi_b;
}
parameters {
  vector[4] pi;
}
model {
  y ~ binomial(n, pi);
  pi ~ beta(pi_a, pi_b);
}
generated quantities {
  vector[2] delta;
  int good[2];
  delta[1] = pi[2] - pi[1];
  delta[2] = pi[4] - pi[3];
  good[1] = int_step(delta[1]);
  good[2] = int_step(delta[2]);
}
\end{minted}

\begin{minted}{R}
# this analysis depends on the order of the data frame
undervote_data <-
  list(y = undervote$undervote,
       n = undervote$n,
       N = nrow(undervote),
       pi_a = rep(1, 4),
       pi_b = rep(1, 4))
\end{minted}

\begin{minted}{R}
undervote_fit <- sampling(undervote_mod, data = undervote_data)
undervote_fit
\end{minted}

Faster estimation of Bayesian models in ecology using Hamiltonian Monte Carlo

